In [4]:
import bagpy
from bagpy import bagreader
import os
import pandas as pd
from dateutil.parser import parse as parsedate
import matplotlib.pyplot as plt
from rosbags.highlevel import AnyReader as RosBagReader
from pathlib import Path
import logging
from dotenv import dotenv_values
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

config = dotenv_values()
data_path = os.path.normpath(config['DATA_PATH'])
os.environ['DATA_PATH'] = data_path
print(f"DATA_PATH is set to: {os.environ['DATA_PATH']}")

def biases_over_days(rat, path, number_of_days):
    if isinstance(rat, str):
        rat = int(rat)

    anim_folder = os.path.join(path, 'NC4%04d' % rat)
    training_folders = get_latest_folders(anim_folder, number_of_days)
    statistics = {}
    for folder in training_folders:
        folder_name = os.path.basename(folder)
        biases = extract_biases (rat, path, folder_name)
        statistics[folder_name] = list(biases)
    return statistics
        
def get_latest_folders(directory, n=7):
   
    subdirs = [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    
    subdirs.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    
    latest_subdirs = subdirs[:n]
    
    return latest_subdirs
    
def extract_biases(rat, date, path=os.environ['DATA_PATH']):
    print(f"Using path: {path}") 
    if isinstance(rat, str):
        rat = int(rat)

    anim_folder = os.path.join(path, 'NC4%04d' % rat)
    print(f"Using path: {path}")
    print(f"Looking in folder: {anim_folder}")

    if '-' in date:  # e.g. date = '24-Jul-24' 
        date = parsedate(date).strftime('%y%m%d')

    date_folder = os.path.join(anim_folder, date)

    print(f"Looking in folder: {date_folder}")
    
    bag_file = None  

    for file in os.listdir(date_folder):
        if file.endswith('.bag'):
            bag_file = os.path.join(date_folder, file)  
            break 

    if bag_file is None:
        raise FileNotFoundError("No .bag file found in the specified folder.")
    
    msg_list = []
    with RosBagReader([Path(bag_file)]) as reader:
        connections = [x for x in reader.connections if x.topic=='/rosout']
        for connection,  timestamp, rawdata in reader.messages(connections=connections):
            msg = reader.deserialize(rawdata, connection.msgtype)
            msg_list.append(msg.msg)

    return msg_list

def filter_start_of_trial_messages(messages):
    keywords = ['Chamber', 'Current trial number', 'START OF TRIAL',
    'Projecting', 'SUCCESS', 'ERROR', 'Right', 'Left']
    return [msg for msg in messages if any(msg.startswith(keyword) for keyword in keywords)]

def remove_specific_message(messages):
    return [msg for msg in messages if msg != 'ERROR_END']

def create_dataframe_from_messages(messages):
    # Calculate the number of rows needed
    num_rows = len(messages) // 6 + (1 if len(messages) % 6 != 0 else 0)
    
    columns =['Number', 'Start Chamber', 'Trial Number', 'Trial', 'Viaual Cue', 'Result', 'Choice']

    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=columns)
    
    # Populate the DataFrame with messages
    for i in range(num_rows):
        start_idx = i * 6
        end_idx = start_idx + 6
        row_data = messages[start_idx:end_idx]
        
        # Fill the remaining columns with empty strings if less than 6 messages in the last row
        while len(row_data) < 6:
            row_data.append('')
        
        df.loc[i] = [i + 1] + row_data
    
    return df

messages = extract_biases(6,'240722')
filtered_masseages = filter_start_of_trial_messages(messages)
final_messages = remove_specific_message(filtered_masseages)
df = create_dataframe_from_messages(final_messages)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DATA_PATH is set to: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Using path: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Using path: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats
Looking in folder: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats\NC40006
Looking in folder: \\10.34.1.59\big_gulp\nc4_rat_data\Maze_Rats\NC40006\240722
    Number       Start Chamber              Trial Number  \
0        1  Chamber 3 selected   Current trial number: 0   
1        2  Chamber 7 selected   Current trial number: 1   
2        3  Chamber 5 selected   Current trial number: 2   
3        4  Chamber 1 selected   Current trial number: 3   
4        5  Chamber 5 selected   Current trial number: 4   
..     ...                 ...                       ...   
56      57  Chamber 3 selected  Current trial number: 56   
57      58  Chamber 7 selected  Current trial number: 57   
58      59  Chamber 5 selected  Current t